# SpaceX Launch Records Dashboard

In [74]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [75]:
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
   
spacex_df.head(10)
#print('Data downloaded and read into a dataframe!')

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,Success,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,Success,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,Success,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,Success,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,Success,0,4535.0,F9 v1.1,v1.1


In [4]:
#!pip install jupyter_dash

In [76]:
spacex_df.head(10)

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,Success,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,Success,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,Success,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,Success,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,Success,0,4535.0,F9 v1.1,v1.1


In [77]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [267]:

# Specific for Jupyter Notebook
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# load Base Data 
data = spacex_df
min_payload = data['Payload Mass (kg)'].min()
max_payload = data['Payload Mass (kg)'].max()


#Convert site into list
Site_list = spacex_df.groupby(['Launch Site']).size().reset_index(name='class count')
Site_list = list(Site_list["Launch Site"])

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                    dcc.Dropdown(
                                        id='site-dropdown',
                                       value='ALL',
                                        options=[      
                                        {'label': 'ALL', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40','value':'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A','value':'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                        ],
                                        placeholder="Select a Launch Site here",
                                        multi=False,
                                        searchable=True),
                                                ]),
                                    
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                               html.Div(dcc.Graph(id='success-pie-chart')),
                                
                               html.Br(),
                               html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                            
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                             
                                
                                
                                
                                        ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))


def pie(site_dropdown):
    
    if site_dropdown == 'ALL':
        fig = px.pie(data, values='class', names='Launch Site', title="All")
    
     
        
    else:
        title_pie = f"Success Launches for site single {site_dropdown}"
        filtered_DD= data[data['Launch Site'] == site_dropdown]
        filtered_LS = filtered_DD.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_LS, values='class count', names='class', title=title_pie)
    return fig


@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])


def render_scatter(site_dropdown,slider_range):
    
    low, high = slider_range
    slide = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    dropdown_scatter=spacex_df[slide]

    if site_dropdown == 'ALL':
        
        #Display all values for the variable
       # dropdown_scatter= spacex_df[spacex_df['Launch Site'] == site_dropdown]
        #dropdown_scatter_all = pd.DataFrame(dropdown_scatter(['Launch Site']))
        title_scatter = f"Payload and Booster Versions for site: {site_dropdown}"
        fig = px.scatter(dropdown_scatter,x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_scatter)
        return fig

    else:
        dropdown_scatter1= dropdown_scatter[dropdown_scatter['Launch Site'] == site_dropdown]
        title_scatter = f"Payload and Booster Versions for site {site_dropdown}"
        fig = px.scatter(dropdown_scatter1, x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_scatter)
        return fig
    
    
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(host="localhost",port=8030, debug=False, dev_tools_ui=False, dev_tools_props_check=False)


Dash app running on http://localhost:8030/


In [ ]:
# Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

# Which site has the largest successful launches?
    #Answer:KSC LC-39A
# Which site has the highest launch success rate?
    # ???
# Which payload range(s) has the highest launch success rate?
    #Answer 2000k-5000k
# Which payload range(s) has the lowest launch success rate?
    #Answer >5000k
# Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
    # Answer = FT
